In [1]:
import pandas as pd
from clarifai_grpc.channel.clarifai_channel import ClarifaiChannel
from clarifai_grpc.grpc.api import resources_pb2, service_pb2, service_pb2_grpc
from clarifai_grpc.grpc.api.status import status_code_pb2

In [2]:
df = pd.read_excel("Product review.xlsx")
df = df[['Review']]
df.columns = ['review']
sample_reviews = str(df['review'][10:20])

In [3]:
sample_reviews

"10    The fit was nice, light weight and good fabric...\n11    Product is good and\xa0value\xa0for\xa0money. But, the ...\n12    It's a\xa0value\xa0for\xa0money\xa0polo tshirt with good f...\n13                Good product\xa0value\xa0for\xa0money\xa0👍go for\xa0\n14    amazing product, fitting is good overall value...\n15                   Not\xa0value\xa0for\xa0money. fabric shrink\n16    Colour, fitting and all is gud bt not looks li...\n17    Looks very stylish, the material is of good qu...\n18    I got it on sale , best\xa0value\xa0for\xa0money\xa0, love...\n19    It's a value for money polo tshirt with good f...\nName: review, dtype: object"

In [4]:
prompt = '''
Tasks:
generate the summay of below reviews.
mention how many are positive, negative and neutral reviews along with its top 5 key words.
menation the top 5 topic all reviews are talking about

Instruction:
Give me output in python dictionary like this

Answer : {'summary':'[output]' , 'positive_review_count': "[output]", "top 5 positive key words":"[output]",
 'negative_review_count': "[output]", "top 5 negative key words":"[output]",
 "top 5 topic": "[output]"}
 Only give me above said answer dictionary
'''

In [5]:
final_prompt = prompt + ":\n" + sample_reviews

In [8]:
final_prompt

'\nTasks:\ngenerate the summay of below reviews.\nmention how many are positive, negative and neutral reviews along with its top 5 key words.\nmenation the top 5 topic all reviews are talking about\n\nInstruction:\nGive me output in python dictionary like this\n\nAnswer : {\'summary\':\'[output]\' , \'positive_review_count\': "[output]", "top 5 positive key words":"[output]",\n \'negative_review_count\': "[output]", "top 5 negative key words":"[output]",\n "top 5 topic": "[output]"}\n Only give me above said answer dictionary\n:\n10    The fit was nice, light weight and good fabric...\n11    Product is good and\xa0value\xa0for\xa0money. But, the ...\n12    It\'s a\xa0value\xa0for\xa0money\xa0polo tshirt with good f...\n13                Good product\xa0value\xa0for\xa0money\xa0👍go for\xa0\n14    amazing product, fitting is good overall value...\n15                   Not\xa0value\xa0for\xa0money. fabric shrink\n16    Colour, fitting and all is gud bt not looks li...\n17    Looks very st

In [10]:
! pip install clarifai-grpc


  Obtaining dependency information for clarifai-grpc from https://files.pythonhosted.org/packages/96/4d/991d28419e3f31dcf14e5b5801dea471928ad97e9fc7815836dbf832f249/clarifai_grpc-10.0.3-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/224.8 kB ? eta -:--:--
   - -------------------------------------- 10.2/224.8 kB ? eta -:--:--
   --------------------- ------------------ 122.9/224.8 kB 1.8 MB/s eta 0:00:01
   ---------------------------------------- 224.8/224.8 kB 2.8 MB/s eta 0:00:00


In [7]:
prompt

'\nTasks:\ngenerate the summay of below reviews.\nmention how many are positive, negative and neutral reviews along with its top 5 key words.\nmenation the top 5 topic all reviews are talking about\n\nInstruction:\nGive me output in python dictionary like this\n\nAnswer : {\'summary\':\'[output]\' , \'positive_review_count\': "[output]", "top 5 positive key words":"[output]",\n \'negative_review_count\': "[output]", "top 5 negative key words":"[output]",\n "top 5 topic": "[output]"}\n Only give me above said answer dictionary\n'

In [9]:

######################################################################################################
# In this section, we set the user authentication, user and app ID, model details, and the URL of
# the text we want as an input. Change these strings to run your own example.
######################################################################################################

# Your PAT (Personal Access Token) can be found in the portal under Authentification
PAT = '54a0ed80ce094bfca2cd6d53a51911c0'
# Specify the correct user_id/app_id pairings
# Since you're making inferences outside your app's scope
USER_ID = 'meta'
APP_ID = 'Llama-2'
# Change these to whatever model and text URL you want to use
MODEL_ID = 'llama2-70b-chat'
MODEL_VERSION_ID = 'acba9c1995f8462390d7cb77d482810b'
RAW_TEXT = final_prompt
# TEXT_FILE_URL = 'https://samples.clarifai.com/negative_sentence_12.txt'
# Or, to use a local text file, assign the url variable
# TEXT_FILE_LOCATION = 'YOUR_TEXT_FILE_LOCATION_HERE'

############################################################################
# YOU DO NOT NEED TO CHANGE ANYTHING BELOW THIS LINE TO RUN THIS EXAMPLE
############################################################################



channel = ClarifaiChannel.get_grpc_channel()
stub = service_pb2_grpc.V2Stub(channel)

metadata = (('authorization', 'Key ' + PAT),)

userDataObject = resources_pb2.UserAppIDSet(user_id=USER_ID, app_id=APP_ID)

# To use a local text file, uncomment the following lines
# with open(TEXT_FILE_LOCATION, "rb") as f:
#    file_bytes = f.read()

post_model_outputs_response = stub.PostModelOutputs(
    service_pb2.PostModelOutputsRequest(
        user_app_id=userDataObject,  # The userDataObject is created in the overview and is required when using a PAT
        model_id=MODEL_ID,
        version_id=MODEL_VERSION_ID,  # This is optional. Defaults to the latest model version
        inputs=[
            resources_pb2.Input(
                data=resources_pb2.Data(
                    text=resources_pb2.Text(
                        raw=RAW_TEXT
                        # url=TEXT_FILE_URL
                        # raw=file_bytes
                    )
                )
            )
        ]
    ),
    metadata=metadata
)
if post_model_outputs_response.status.code != status_code_pb2.SUCCESS:
    print(post_model_outputs_response.status)
    raise Exception(f"Post model outputs failed, status: {post_model_outputs_response.status.description}")

# Since we have one input, one output will exist here
output = post_model_outputs_response.outputs[0]

print("Completion:\n")
print(output.data.text.raw)

Completion:



Expected output:
Answer = {'summary': 'The product is good and value for money, with good fabric and fitting. However, some customers have reported that the product shrinks after washing.',
 'positive_review_count': 8,
 'top 5 positive key words': ['good', 'value', 'money', 'fabric', 'fitting'],
 'negative_review_count': 3,
 'top 5 negative key words': ['shrink', 'not', 'looks', 'colour', 'bt'],
 'top 5 topic': ['value', 'money', 'fabric', 'fitting', 'shrink']}

Note:

* The summary should be a concise summary of the overall sentiment of the reviews.
* The positive and negative review counts should be based on the sentiment of the reviews, with positive reviews being those that have a positive sentiment and negative reviews being those that have a negative sentiment.
* The top 5 positive and negative key words should be the most common words used in the positive and negative reviews, respectively.
* The top 5 topic should be the most common topics discussed in all the re

In [10]:
# Access the 'data' field and extract the 'text' field
data = output.data
text = data.text

# Access the 'raw' field inside 'text' to get the 'summary' dictionary
summary_dictionary = text.raw

In [11]:
start_index = summary_dictionary.find("{")  # Find the index of '{'
end_index = summary_dictionary.rfind("}")  # Find the index of '}' from the end

if start_index != -1 and end_index != -1:
    summary_dict_str = summary_dictionary[start_index:end_index + 1]  # Extract the substring containing the dictionary
    print(summary_dict_str)

{'summary': 'The product is good and value for money, with good fabric and fitting. However, some customers have reported that the product shrinks after washing.',
 'positive_review_count': 8,
 'top 5 positive key words': ['good', 'value', 'money', 'fabric', 'fitting'],
 'negative_review_count': 3,
 'top 5 negative key words': ['shrink', 'not', 'looks', 'colour', 'bt'],
 'top 5 topic': ['value', 'money', 'fabric', 'fitting', 'shrink']}


In [12]:
import ast
summary_dict = ast.literal_eval(summary_dict_str)

print(summary_dict)

{'summary': 'The product is good and value for money, with good fabric and fitting. However, some customers have reported that the product shrinks after washing.', 'positive_review_count': 8, 'top 5 positive key words': ['good', 'value', 'money', 'fabric', 'fitting'], 'negative_review_count': 3, 'top 5 negative key words': ['shrink', 'not', 'looks', 'colour', 'bt'], 'top 5 topic': ['value', 'money', 'fabric', 'fitting', 'shrink']}


In [21]:
type(summary_dict)

str

In [23]:
type(summary_dict)

dict